<center><strong><h1>Universidade Federal de Alagoas</h1></strong></center>
<center><strong><h1>IC - Instituto de Computação</h1></strong></center>
<center><strong><h2>Modelagem e Controle de Robos - 2022.1</h2></strong></center>

---

**Professores**: Ícaro Araújo e Glauber Leite

---

<center><strong><h2>Controle por Força</h2></strong></center>

<center><strong><h3>1. Setup: </h3></strong></center>

In [29]:
#Imports
import numpy as np
from IPython import display

from sympy.physics.mechanics import dynamicsymbols, init_vprinting
from sympy import Function, simplify, Matrix, collect, factor, Wild
from sympy import sin as s, cos as c, symbols as symbol, diff as df

In [30]:
init_vprinting()

In [31]:
#Variáveis simbólicas
t, g, pi = symbol('t'), symbol('g'), symbol('\\pi')
L2, L3 = symbol('L_2'), symbol('L_3')
m2, m3 = symbol('m_2'), symbol('m_3')
t1, t2, t3 = Function('\\theta_1')(t), Function('\\theta_2')(t), Function('\\theta_3')(t)

<center><strong><h3>2. Jacobiana Analítica </h3></strong></center>

### Matrizes de transformação até o efetuador:

Do *frame* 0 até o *frame* 1, temos:

\begin{gather*} 
    _{0}^{1}{T} = 
    \begin{bmatrix}
    c\theta_1 &  0 & -s\theta_1 & 0\\ 
    s\theta_1 &  0 & c\theta_1  & 0\\ 
    0         & -1 & 0          & 0.11\\ 
    0         &  0 & 0          & 1
    \end{bmatrix}
\end{gather*} 

Do *frame* 1 até o *frame* 2, temos:

\begin{gather*} 
    _{1}^{2}{T} = 
    \begin{bmatrix}
    c\left ( \frac{2 \theta_2 - \pi}{2} \right ) & -s\left ( \frac{2 \theta_2 - \pi}{2} \right ) & 0 & 0.125c\left ( \frac{2 \theta_2 - \pi}{2} \right )\\ 
    s\left ( \frac{2 \theta_2 - \pi}{2} \right ) & c\left ( \frac{2 \theta_2 - \pi}{2} \right )  & 0 & 0.125s\left ( \frac{2 \theta_2 - \pi}{2} \right )\\ 
    0                                            & 0                                             & 1 & 0\\ 
    0                                            & 0                                             & 0 & 1
    \end{bmatrix}
\end{gather*} 

Do *frame* 2 até o *frame* 3, temos:

\begin{gather*} 
    _{2}^{3}{T} = 
    \begin{bmatrix}
    c\left ( \frac{2 \theta_3 + \pi}{2} \right ) & -s\left ( \frac{2 \theta_3 + \pi}{2} \right)  & 0 & 0.096c\left ( \frac{2 \theta_3 + \pi}{2} \right )\\ 
    s\left ( \frac{2 \theta_3 + \pi}{2} \right ) & c\left ( \frac{2 \theta_3 + \pi}{2} \right )  & 0 & 0.096s\left ( \frac{2 \theta_3 + \pi}{2} \right )\\ 
    0                                            & 0                                             & 1 & 0\\ 
    0                                            & 0                                             & 0 & 1
    \end{bmatrix}
\end{gather*} 

Do *frame* 3 até o *frame* 4, temos:

\begin{gather*} 
    _{3}^{4}{T} = 
    \begin{bmatrix}
    0 & 0 & 1 & 0\\ 
    1 & 0 & 0 & -0.0275\\ 
    0 & 1 & 0 & 0\\ 
    0 & 0 & 0 & 1
    \end{bmatrix}
\end{gather*} 

Do *frame* 4 até o *frame* 5, temos:

\begin{gather*} 
    _{4}^{5}{T} = 
    \begin{bmatrix}
    1 & 0 & 0 & 0\\ 
    0 & 1 & 0 & 0\\ 
    0 & 0 & 1 & 0.065\\ 
    0 & 0 & 0 & 1
    \end{bmatrix}
\end{gather*} 

Dessa forma:

In [32]:
#Matrizes de trans. homogênea

T0_1 = np.array([[c(t1), 0, -s(t1), 0,],
                 [s(t1), 0,  c(t1), 0,],
                 [0,       -1,        0,  0.11,],
                 [0,        0,        0,  1]])

T1_2 = np.array([[c((2*t2-pi)/2),-s((2*t2-pi)/2),  0, 0.125*c((2*t2-pi)/2)],
                 [s((2*t2-pi)/2), c((2*t2-pi)/2),  0, 0.125*s((2*t2-pi)/2)],
                 [             0,              0,  1,                    0],
                 [             0,              0,  0,                    1]])

T2_3 = np.array([[c((2*t3+pi)/2),-s((2*t3+pi)/2),  0, 0.096*c((2*t3+pi)/2)],
                 [s((2*t3+pi)/2), c((2*t3+pi)/2),  0, 0.096*s((2*t3+pi)/2)],
                 [0,           0, 1,                                     0],
                 [0,           0, 0,                                     1]])

T3_4 = np.array([[0, 0,  1,       0],
                 [1, 0,  0, -0.0275],
                 [0, 1,  0,       0],
                 [0, 0,  0,       1]])

T4_5 = np.array([[1, 0, 0,     0],
                 [0, 1, 0,     0],
                 [0, 0, 1, 0.065],
                 [0, 0, 0,     1]])

### Portanto, temos a seguinte matriz de transformação que leva do *frame* 0 até o *frame* 5:

\begin{gather*} 
    _{0}^{5}{T} = _{0}^{1}{T}\cdot_{1}^{2}{T}\cdot_{2}^{3}{T}\cdot_{3}^{4}{T}\cdot_{4}^{5}{T}
\end{gather*} 

In [33]:
#Cinemática direta
FKine = T0_1 @ T1_2 @ T2_3 @ T3_4 @ T4_5
Matrix(simplify(FKine))

⎡                                                                             
⎢-1.0⋅sin(\theta₂ + \theta₃)⋅cos(\theta₁)  -1.0⋅sin(\theta₁)  1.0⋅cos(\theta₂ 
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢-1.0⋅sin(\theta₂ + \theta₃)⋅sin(\theta₁)  1.0⋅cos(\theta₁)   1.0⋅sin(\theta₁)
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢      -1.0⋅cos(\theta₂ + \theta₃)                 0                -1.0⋅sin(\
⎢                                                                             
⎢                                                                             
⎣                   0                              0

In [34]:
#P = [x , y, z]^T
T_x = simplify(FKine[0][3])
T_y = simplify(FKine[1][3])
T_z = simplify(FKine[2][3])

### Por fim, obtemos a seguinte matriz Jacobiana analítica:

In [35]:
#Matriz Jacobiana analítica com as derivadas parciais
Matrix([[df(T_x, t1), df(T_x, t2), df(T_x, t3)],
        [df(T_y, t1), df(T_y, t2), df(T_z, t3)],
        [df(T_z, t1), df(T_z, t2), df(T_z, t3)]])

⎡ ⎛                                         ⎛\pi          ⎞                   
⎢-⎜0.0275⋅sin(\theta₂ + \theta₃) + 0.125⋅cos⎜─── - \theta₂⎟ + 0.161⋅cos(\theta
⎢ ⎝                                         ⎝ 2           ⎠                   
⎢                                                                             
⎢⎛                                         ⎛\pi          ⎞                    
⎢⎜0.0275⋅sin(\theta₂ + \theta₃) + 0.125⋅cos⎜─── - \theta₂⎟ + 0.161⋅cos(\theta₂
⎢⎝                                         ⎝ 2           ⎠                    
⎢                                                                             
⎢                                                                             
⎢                                                   0                         
⎣                                                                             

            ⎞               ⎛         ⎛\pi          ⎞                         
₂ + \theta₃)⎟⋅sin(\theta₁)  ⎜0.125⋅sin⎜─── - \theta

<center><strong><h3>3. Lagrangiana Analítica </h3></strong></center>

### Partindo do seguinte diagrama:

<image src= https://raw.githubusercontent.com/Tayco110/Robotics/main/mcdr/2.png>

### Podemos determinar a seguinte tabela de parâmetros DH para os centros de massa:

<image src= https://raw.githubusercontent.com/Tayco110/Robotics/main/mcdr/1.png>

### Montando as devidas matrizes de transformação, podemos obter as coodernadas dos pontos $x$, $y$ e $z$ de cada centro de massa $m$.

### Posição do ponto 2:

In [36]:
#Posição do ponto 2
P2 = [[L2 * c(t1) * c(t2)],
      [L2 * s(t1) * c(t2)],
      [-L2 * s(t2)]]
Matrix(P2)

⎡L₂⋅cos(\theta₁)⋅cos(\theta₂)⎤
⎢                            ⎥
⎢L₂⋅sin(\theta₁)⋅cos(\theta₂)⎥
⎢                            ⎥
⎣      -L₂⋅sin(\theta₂)      ⎦

### Velocidade do ponto 2:

In [37]:
#Velocidade do ponto 2
x2_dot , y2_dot, z2_dot = df(P2[0][0],t), df(P2[1][0],t), df(P2[2][0],t)
Matrix([x2_dot, y2_dot, z2_dot])

⎡-L₂⋅sin(\theta₁)⋅cos(\theta₂)⋅\thėta₁ - L₂⋅sin(\theta₂)⋅cos(\theta₁)⋅\thėta₂⎤
⎢                                                                            ⎥
⎢-L₂⋅sin(\theta₁)⋅sin(\theta₂)⋅\thėta₂ + L₂⋅cos(\theta₁)⋅cos(\theta₂)⋅\thėta₁⎥
⎢                                                                            ⎥
⎣                          -L₂⋅cos(\theta₂)⋅\thėta₂                          ⎦

### Posição do ponto 3:

In [38]:
#Posição do ponto 3
P3 = [[L3 * c(t1) * (c(t2)*c(t3) - s(t2)*s(t3)) + 2*P2[0][0]],
      [L3 * s(t1) * (c(t2)*c(t3) - s(t2)*s(t3)) + 2*P2[1][0]],
      [-L3 * (s(t2)*c(t3) + c(t2)*s(t3)) + 2*P2[2][0]]]
Matrix(P3)

⎡2⋅L₂⋅cos(\theta₁)⋅cos(\theta₂) + L₃⋅(-sin(\theta₂)⋅sin(\theta₃) + cos(\theta₂
⎢                                                                             
⎢2⋅L₂⋅sin(\theta₁)⋅cos(\theta₂) + L₃⋅(-sin(\theta₂)⋅sin(\theta₃) + cos(\theta₂
⎢                                                                             
⎣             -2⋅L₂⋅sin(\theta₂) - L₃⋅(sin(\theta₂)⋅cos(\theta₃) + sin(\theta₃

)⋅cos(\theta₃))⋅cos(\theta₁)⎤
                            ⎥
)⋅cos(\theta₃))⋅sin(\theta₁)⎥
                            ⎥
)⋅cos(\theta₂))             ⎦

### Velocidade do ponto 3:

In [39]:
#Velocidade do ponto 3
x3_dot , y3_dot, z3_dot = df(P3[0][0],t), df(P3[1][0],t), df(P3[2][0],t)
Matrix([x3_dot, y3_dot, z3_dot])

⎡-2⋅L₂⋅sin(\theta₁)⋅cos(\theta₂)⋅\thėta₁ - 2⋅L₂⋅sin(\theta₂)⋅cos(\theta₁)⋅\th
⎢                                                                             
⎢-2⋅L₂⋅sin(\theta₁)⋅sin(\theta₂)⋅\thėta₂ + 2⋅L₂⋅cos(\theta₁)⋅cos(\theta₂)⋅\th
⎢                                                                             
⎣                                                                           -2

ėta₂ - L₃⋅(-sin(\theta₂)⋅sin(\theta₃) + cos(\theta₂)⋅cos(\theta₃))⋅sin(\theta
                                                                              
ėta₁ + L₃⋅(-sin(\theta₂)⋅sin(\theta₃) + cos(\theta₂)⋅cos(\theta₃))⋅cos(\theta
                                                                              
⋅L₂⋅cos(\theta₂)⋅\thėta₂ - L₃⋅(-sin(\theta₂)⋅sin(\theta₃)⋅\thėta₂ - sin(\the

₁)⋅\thėta₁ + L₃⋅(-sin(\theta₂)⋅cos(\theta₃)⋅\thėta₂ - sin(\theta₂)⋅cos(\thet
                                                                              
₁)⋅\thėta₁ + L₃⋅(-sin(\theta₂)⋅cos(\theta₃)⋅\thė

Na Formulação de Lagrange, um conjunto de   variáveis $q_i$, $i$ = $1,...,n$, denominadas coordenadas generalizadas, são escolhidas para descrever a posição da junta de um manipulador com $n$ \textit{DOF}, o Lagrangiano de um sistema dinâmico pode ser definido como uma função de coordenadas(SCIAVICCO; SICILIANO, 2012):
\begin{gather*}
        \mathcal{L} = \mathcal{T} - \mathcal{U}
\end{gather*}
    onde $\mathcal{T}$ e $\mathcal{U}$ denotam, respectivamente, a energia cinética e a energia potencial do sistema. Dessa forma, As equações Lagrangianas são expressas na forma:
\begin{gather*}
        \frac{d}{dt} \frac{\partial \mathcal{L}}{\partial \dot{q_i}} - \frac{\partial \mathcal{L}}{\partial {q_i}} = \xi_i
\end{gather*}
    onde $\xi_i$ é a força generalizada associada à coordenada generalizada $q_i$. Dessa forma:

In [40]:
#Lagrangenana
T = (m2 / 2) * (x2_dot**2 + y2_dot**2 + z2_dot**2) + (m3 / 2) * (x3_dot**2 + y3_dot**2 + z3_dot**2)
U = -m2 * g * L2 * s(t2) - m3 * g * (L3 * s(t2 + t3) + 2 * L2 * s(t2))
L = T - U

simplify(L)

  2    ⎛   2                 2          2⎞                                    
L₂ ⋅m₂⋅⎝cos (\theta₂)⋅\thėta₁  + \thėta₂ ⎠                                  
────────────────────────────────────────── + L₂⋅g⋅m₂⋅sin(\theta₂) + g⋅m₃⋅(2⋅L₂
                    2                                                         

                                                ⎛    2    2                 2 
                                               m₃⋅⎝4⋅L₂ ⋅cos (\theta₂)⋅\thėta
⋅sin(\theta₂) + L₃⋅sin(\theta₂ + \theta₃)) + ─────────────────────────────────
                                                                              

      2        2                                                      2       
₁  + 4⋅L₂ ⋅\thėta₂  + 4⋅L₂⋅L₃⋅cos(\theta₂ + \theta₃)⋅cos(\theta₂)⋅\thėta₁  +
──────────────────────────────────────────────────────────────────────────────
                                                                              

                        2                        

Logo, os valores de $\tau$ para as juntas são:

In [41]:
T1 = df(df(L, df(t1, t)), t) + df(L, t1); simplify(T1)  #Torque na junta 1

  2    ⎛                                  cos(2⋅\theta₂)⋅\thëta₁   \thëta₁⎞ 
L₂ ⋅m₂⋅⎜-sin(2⋅\theta₂)⋅\thėta₁⋅\thėta₂ + ────────────────────── + ───────⎟ 
       ⎝                                            2                 2   ⎠   

                                                                              
+ m₃⋅(2⋅L₂⋅cos(\theta₂) + L₃⋅cos(\theta₂ + \theta₃))⋅(-4⋅L₂⋅sin(\theta₂)⋅\thė
                                                                              

                                                                              
ta₁⋅\thėta₂ + 2⋅L₂⋅cos(\theta₂)⋅\thëta₁ - 2⋅L₃⋅sin(\theta₂ + \theta₃)⋅\thėt
                                                                              

                                                                              
a₁⋅\thėta₂ - 2⋅L₃⋅sin(\theta₂ + \theta₃)⋅\thėta₁⋅\thėta₃ + L₃⋅cos(\theta₂ +
                                                                              

           
 \theta₃)⋅\thëta₁)
         

In [42]:
T2 = df(df(L, df(t2, t)), t) + df(L, t2); simplify(T2) #Torque na junta 2 

    2                          2                                              
  L₂ ⋅m₂⋅sin(2⋅\theta₂)⋅\thėta₁      2                                       
- ────────────────────────────── + L₂ ⋅m₂⋅\thëta₂ + L₂⋅g⋅m₂⋅cos(\theta₂) + g⋅
                2                                                             

                                                       ⎛                      
                                                        ⎜    2                
m₃⋅(2⋅L₂⋅cos(\theta₂) + L₃⋅cos(\theta₂ + \theta₃)) - m₃⋅⎜2⋅L₂ ⋅sin(2⋅\theta₂) 
                                                       ⎝                      

                                      2                           ⎞           
                                     L₃ ⋅sin(2⋅\theta₂ + 2⋅\theta₃)⎟        2 
+ 2⋅L₂⋅L₃⋅sin(2⋅\theta₂ + \theta₃) + ──────────────────────────────⎟⋅\thėta₁ 
                                                  2               ⎠           

                                                 

In [43]:
T3 = df(df(L, df(t3, t)), t) + df(L, t3); simplify(T3)  #Torque na junta 3

      ⎛                                                                       
      ⎜                                     2                          2      
L₃⋅m₃⋅⎜- L₂⋅sin(2⋅\theta₂ + \theta₃)⋅\thėta₁  - L₂⋅sin(\theta₃)⋅\thėta₁  - 2
      ⎝                                                                       

                                                                              
                      2                                                       
⋅L₂⋅sin(\theta₃)⋅\thėta₂  - 4⋅L₂⋅sin(\theta₃)⋅\thėta₂⋅\thėta₃ + 2⋅L₂⋅cos(\t
                                                                              

                                                 2                            
            L₃⋅sin(2⋅\theta₂ + 2⋅\theta₃)⋅\thėta₁                            
heta₃)⋅\thëta₂ - ────────────────────────────────────── + L₃⋅\thëta₂ + L₃⋅\t
                              2                                               

                         ⎞
                      

<center><strong><h3>4. Dinâmica no espaço de Trabalho </h3></strong></center>

Uma vez que temos os modelos analíticos de nossa Jacobiana e também da Lagrangiana, podemos montar a seguinte relação:

\begin{gather*}
{J}^{-T} \tau = {J}^{-T} M {J}^{-1} \dot{\mathcal{V}} - {J}^{-T}M {J}^{-1}\dot{J}M {J}^{-1} \dot{\mathcal{V}} + {J}^{-T}h\left ( \theta,{J}^{-T} \mathcal{V} \right )
\end{gather*}

A equação acima também pode ser escrita como:

\begin{gather*}
\mathcal{F} = \Lambda \left ( \theta \right ) \dot{\mathcal{V}} + \eta \left ( \theta, \mathcal{V} \right )
\end{gather*}

onde:

\begin{gather*}
\mathcal{F} = {J}^{-T} \tau
\end{gather*}

\begin{gather*}
\Lambda \left ( \theta \right ) \dot{\mathcal{V}} = {J}^{-T} M {J}^{-1} \dot{\mathcal{V}} - {J}^{-T}M {J}^{-1}\dot{J}M {J}^{-1} \dot{\mathcal{V}}
\end{gather*}

\begin{gather*}
\eta \left ( \theta, \mathcal{V} \right ) = {J}^{-T}h\left ( \theta,{J}^{-T} \mathcal{V} \right )
\end{gather*}